# Myth Museum - Interactive Video Generator

This notebook allows you to create myth-busting YouTube Shorts videos interactively.

## Workflow
1. Enter your topic
2. Select visual style and narrative arc
3. Generate and preview script
4. Preview image prompts
5. Generate images (with preview)
6. Regenerate any images you don't like
7. Generate final video

## Step 1: Environment Setup

In [ ]:
import sys
import os
from pathlib import Path
from IPython.display import display, Image, HTML, Video
import ipywidgets as widgets

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
os.chdir(project_root)

print(f"Working directory: {Path.cwd()}")

# Import project modules
from pipeline.storyboard_templates import (
    STYLE_TEMPLATES, NARRATIVE_ARCS,
    get_arc_with_prompts, detect_arc_type
)
from pipeline.character_library import (
    detect_character_from_topic, get_character_for_prompt
)
from pipeline.visual_director import StoryboardDirector
from pipeline.generate_scripts import generate_myth_script
from pipeline.image_providers.vertex_imagen import ImageProviderWithFallback
from pipeline.tts import GoogleTTSProvider, get_audio_duration
from pipeline.folder_naming import generate_folder_name

print("Modules loaded successfully!")

## Step 2: Enter Your Topic

In [ ]:
# Topic input
topic_input = widgets.Text(
    value='Da Vinci Mona Lisa myth',
    placeholder='Enter your topic...',
    description='Topic:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

# Optional: myth and truth for context
myth_input = widgets.Textarea(
    value='',
    placeholder='Optional: The common myth people believe...',
    description='Myth:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%', height='60px')
)

truth_input = widgets.Textarea(
    value='',
    placeholder='Optional: The actual truth...',
    description='Truth:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%', height='60px')
)

display(topic_input, myth_input, truth_input)

# Store values
TOPIC = topic_input.value
print(f"\nCurrent topic: {TOPIC}")

## Step 3: Select Style and Narrative Arc

In [ ]:
# Get current topic value
TOPIC = topic_input.value
MYTH = myth_input.value
TRUTH = truth_input.value

# Style selector
style_options = [(f"{v.display_name} - {v.description[:40]}...", k) for k, v in STYLE_TEMPLATES.items()]
style_dropdown = widgets.Dropdown(
    options=style_options,
    value='oil_painting_cartoon',
    description='Visual Style:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

# Arc selector
arc_options = [(f"{arc_id} ({len(scenes)} scenes)", arc_id) for arc_id, scenes in NARRATIVE_ARCS.items()]
detected_arc = detect_arc_type(TOPIC)
arc_dropdown = widgets.Dropdown(
    options=arc_options,
    value=detected_arc,
    description='Narrative Arc:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

# Quality selector
quality_dropdown = widgets.Dropdown(
    options=[
        ('High (Imagen 3) - Best quality, slow', 'high'),
        ('Standard (Imagen @006) - Good quality, fast', 'standard'),
        ('Fallback (Stock photos) - Quick testing', 'fallback'),
    ],
    value='high',
    description='Image Quality:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

display(style_dropdown, arc_dropdown, quality_dropdown)

# Auto-detect character
char_profile = detect_character_from_topic(TOPIC)
if char_profile:
    print(f"\nAuto-detected character: {char_profile.name}")
    print(f"Description: {char_profile.short_description}")
else:
    print(f"\nNo predefined character found for topic. Using generic description.")

print(f"\nAuto-detected arc: {detected_arc}")

## Step 4: Generate or Enter Script

Enter your script manually or use LLM to generate one:

In [ ]:
# Get current values
TOPIC = topic_input.value
MYTH = myth_input.value
TRUTH = truth_input.value
STYLE = style_dropdown.value
ARC = arc_dropdown.value
QUALITY = quality_dropdown.value

print(f"Topic: {TOPIC}")
print(f"Style: {STYLE}, Arc: {ARC}, Quality: {QUALITY}")
print("\n" + "="*50 + "\n")

# Default script template
DEFAULT_SCRIPT = f"""Think {TOPIC} is just an old myth?

Here's what most people believe...

But recent research reveals something surprising.

The evidence shows a completely different story.

Scientists have discovered the truth.

And the real answer might blow your mind.

Follow for more myth-busting content!
"""

# Editable script
script_textarea = widgets.Textarea(
    value=DEFAULT_SCRIPT,
    description='Script:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%', height='300px')
)

display(script_textarea)

# Word count
word_count = len(DEFAULT_SCRIPT.split())
estimated_duration = word_count / 2.5
print(f"\nWord count: {word_count} (~{estimated_duration:.0f} seconds)")

## Step 5: Preview Image Prompts

In [ ]:
# Get current values
TOPIC = topic_input.value
STYLE = style_dropdown.value
ARC = arc_dropdown.value
SCRIPT = script_textarea.value

# Get character description
CHARACTER = get_character_for_prompt(TOPIC)

# Generate prompts using storyboard templates
PROMPTS = get_arc_with_prompts(
    arc_type=ARC,
    style_id=STYLE,
    topic=TOPIC,
    character=CHARACTER,
)

print(f"Generated {len(PROMPTS)} image prompts:\n")

# Create editable prompt widgets
prompt_widgets = []
for i, p in enumerate(PROMPTS):
    scene_name = p.get('scene_name', f'Scene {i+1}')
    print(f"[{p['segment']}] {scene_name}")
    
    w = widgets.Textarea(
        value=p['prompt'],
        description=f"{p['segment']}:",
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='100%', height='80px')
    )
    prompt_widgets.append(w)
    display(w)
    print()

## Step 6: Generate Images

This will generate all 6 images using the selected quality setting.

In [ ]:
import json
import asyncio
from datetime import datetime

# Get current values
TOPIC = topic_input.value
SCRIPT = script_textarea.value
QUALITY = quality_dropdown.value

# Create output folder
folder_name = generate_folder_name("interactive", TOPIC)
OUTPUT_DIR = Path("outputs/shorts") / folder_name
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
(OUTPUT_DIR / "backgrounds").mkdir(exist_ok=True)

print(f"Output folder: {OUTPUT_DIR}")
print(f"Quality: {QUALITY}")
print("\nGenerating images...\n")

# Update prompts from widgets
for i, w in enumerate(prompt_widgets):
    PROMPTS[i]['prompt'] = w.value

# Save prompts
with open(OUTPUT_DIR / "visual_briefs.json", "w", encoding="utf-8") as f:
    json.dump(PROMPTS, f, indent=2, ensure_ascii=False)

# Save script
(OUTPUT_DIR / "voiceover.txt").write_text(SCRIPT, encoding="utf-8")

# Generate images
provider = ImageProviderWithFallback()

async def generate_all_images():
    results = []
    for i, p in enumerate(PROMPTS):
        output_path = OUTPUT_DIR / "backgrounds" / f"bg_{i+1}.jpg"
        print(f"[{i+1}/{len(PROMPTS)}] Generating bg_{i+1}...")
        
        result = await provider.generate_image(
            prompt=p['prompt'],
            output_path=output_path,
            fallback_keyword=p.get('fallback_keyword', TOPIC),
            quality=QUALITY,
        )
        
        if result.success:
            print(f"    OK ({result.source}, {result.latency_ms}ms)")
        else:
            print(f"    FAILED: {result.error}")
        
        results.append(result)
    return results

# Run async
loop = asyncio.get_event_loop()
IMAGE_RESULTS = loop.run_until_complete(generate_all_images())

print(f"\nGenerated {sum(1 for r in IMAGE_RESULTS if r.success)}/{len(PROMPTS)} images")

## Step 7: Preview Generated Images

In [ ]:
from PIL import Image as PILImage
import matplotlib.pyplot as plt

# Display all images in a grid
images = sorted((OUTPUT_DIR / "backgrounds").glob("bg_*.jpg"))

if images:
    n_cols = 3
    n_rows = (len(images) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 8 * n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes
    
    for i, img_path in enumerate(images):
        img = PILImage.open(img_path)
        axes[i].imshow(img)
        axes[i].set_title(img_path.name)
        axes[i].axis('off')
    
    # Hide empty axes
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("No images found! Run the previous cell first.")

## Step 8: Regenerate Single Image (Optional)

If you're not happy with an image, change `REGENERATE_NUM` and run this cell:

In [ ]:
# Which image to regenerate? (1-6)
REGENERATE_NUM = 1

# Get the prompt (can also edit it directly here)
prompt_to_use = prompt_widgets[REGENERATE_NUM - 1].value

print(f"Regenerating image {REGENERATE_NUM}...")
print(f"Prompt: {prompt_to_use[:100]}...")

async def regenerate_image():
    output_path = OUTPUT_DIR / "backgrounds" / f"bg_{REGENERATE_NUM}.jpg"
    
    # Backup old image
    if output_path.exists():
        backup_path = OUTPUT_DIR / "backgrounds" / f"bg_{REGENERATE_NUM}_backup.jpg"
        import shutil
        shutil.copy(output_path, backup_path)
        print(f"Backed up to {backup_path.name}")
    
    result = await provider.generate_image(
        prompt=prompt_to_use,
        output_path=output_path,
        fallback_keyword=TOPIC,
        quality=QUALITY,
    )
    
    if result.success:
        print(f"OK! Source: {result.source}, Latency: {result.latency_ms}ms")
        # Display the new image
        img = PILImage.open(output_path)
        plt.figure(figsize=(6, 10))
        plt.imshow(img)
        plt.title(f"bg_{REGENERATE_NUM}.jpg (NEW)")
        plt.axis('off')
        plt.show()
    else:
        print(f"FAILED: {result.error}")

loop.run_until_complete(regenerate_image())

## Step 9: Generate Final Video

This will:
1. Generate TTS audio from your script
2. Create subtitles (SRT + ASS)
3. Render slideshow with Ken Burns effect
4. Combine audio, video, and subtitles

In [ ]:
import subprocess
from pipeline.generate_short import ShortVideoGenerator
from pipeline.tts import script_to_srt

SCRIPT = script_textarea.value

print("Generating final video...\n")

# 1. Generate TTS audio
print("[1/4] Generating voiceover audio...")
audio_path = OUTPUT_DIR / "voiceover.mp3"

async def generate_audio():
    tts = GoogleTTSProvider()
    await tts.synthesize(
        text=SCRIPT,
        output_path=audio_path,
        voice="en-US-Casual-K",
    )

loop.run_until_complete(generate_audio())
duration = get_audio_duration(audio_path)
print(f"    Audio duration: {duration:.1f}s")

# 2. Create SRT subtitles
print("[2/4] Creating subtitles...")
srt_path = OUTPUT_DIR / "captions.srt"
script_to_srt(SCRIPT, srt_path, duration)

# 3. Create ASS subtitles with series marker
print("[3/4] Converting to ASS format...")
ass_path = OUTPUT_DIR / "captions.ass"
generator = ShortVideoGenerator()
generator._create_ass_with_series_marker(
    srt_path=srt_path,
    ass_path=ass_path,
    series_text="Interactive #1",
    duration=duration,
)

# 4. Render video
print("[4/4] Rendering final video...")
images = sorted((OUTPUT_DIR / "backgrounds").glob("bg_*.jpg"))
slideshow_path = OUTPUT_DIR / "background.mp4"
final_path = OUTPUT_DIR / "final.mp4"

# Create slideshow
generator._create_slideshow(
    images=images,
    output_path=slideshow_path,
    duration=duration,
    use_ken_burns=True,
    use_crossfade=True,
)

# Render final video
generator._render_final(
    slideshow_path=slideshow_path,
    audio_path=audio_path,
    ass_path=ass_path,
    output_path=final_path,
)

if final_path.exists():
    size_mb = final_path.stat().st_size / (1024 * 1024)
    print(f"\n{'='*50}")
    print(f"SUCCESS!")
    print(f"{'='*50}")
    print(f"Video: {final_path}")
    print(f"Size: {size_mb:.1f} MB")
    print(f"Duration: {duration:.1f}s")
else:
    print("\nFailed to create video!")

In [ ]:
# Display the video path and open option
final_path = OUTPUT_DIR / "final.mp4"

if final_path.exists():
    print(f"Video location: {final_path.absolute()}")
    print(f"\nTo play the video, run:")
    print(f'start "{final_path.absolute()}"')
    
    # Try to display in notebook
    try:
        display(Video(str(final_path), width=400, embed=True))
    except Exception as e:
        print(f"\n(Video preview not supported: {e})")
        print("Please open the video file directly from the output folder.")
else:
    print("Video not found! Run the previous cell first.")

---

## Summary

Your generated files:

In [ ]:
print(f"Output folder: {OUTPUT_DIR.absolute()}")
print("\nFiles:")
for f in sorted(OUTPUT_DIR.glob("*")):
    if f.is_file():
        size = f.stat().st_size / 1024
        unit = "KB"
        if size > 1024:
            size = size / 1024
            unit = "MB"
        print(f"  {f.name}: {size:.1f} {unit}")

print("\nBackgrounds:")
for f in sorted((OUTPUT_DIR / "backgrounds").glob("*.jpg")):
    size = f.stat().st_size / 1024
    print(f"  {f.name}: {size:.1f} KB")

print("\n" + "="*50)
print("Done! You can find your video at:")
print(f"{final_path.absolute()}")